In [15]:
#!pip install tensorflow
#!pip install pandas as pd
#!pip install numpy as np
#!pip install --upgrade pip

In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv("hero_selection_final.csv")

In [18]:
df1 = pd.read_csv("hero_names.csv")

In [19]:
matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])

In [20]:
df2 = pd.read_csv("hero_selection.csv")
#df2

In [21]:
matrix3 =np.array(df2.loc[:,['target']])

In [22]:
'''Category prediction of hero5 will contain the data of hero1, hero2, hero3, hero4.'''
matrix4 =np.array(df2.loc[:,[ 'Hero1','Hero2','Hero3','Hero4']])
matrix5 = np.array(df2.loc[:,[ 'Hero5']])
'''Category prediction of hero2 will contain the data of hero1.'''
matrix43 =np.array(df2.loc[:,[ 'Hero1']])
matrix53 = np.array(df2.loc[:,[ 'Hero2']])
'''Category prediction of hero3 will contain the data of hero1, hero2.'''
matrix42 =np.array(df2.loc[:,[ 'Hero1','Hero2']])
matrix52 = np.array(df2.loc[:,[ 'Hero3']])
'''Category prediction of hero4 will contain the data of hero1, hero2, hero3'''
matrix41 =np.array(df2.loc[:,[ 'Hero1','Hero2','Hero3']])
matrix51 = np.array(df2.loc[:,[ 'Hero4']])

In [23]:
''' target variable contains the category of next hero picked in the next pick. '''
def test_train(matrix4, matrix5):
    feature_matrixes = matrix4
    target_features = matrix5
    train_data = feature_matrixes[0:40000]
    test_data = feature_matrixes[40000:50000]
    train_target = target_features[0:40000]
    test_target = target_features[40000:50000]
    
    #train target preparation for categorical data
    train_target.reshape(40000,)
    final_index=[]
    for i in range(40000):
        hero = int(train_target[i][0])
        indexed = matrix2[hero-1]
      
        if indexed[1]==1:
            final_index.append(1)
        elif indexed[2]==1:
            final_index.append(2)
        elif indexed[3]==1:
            final_index.append(3)
    final_index = np.array(final_index)
    
    #test target preparation for categorical data
    test_target.reshape(10000,)
    final_index_test=[]
    for i in range(10000):
        hero = int(test_target[i][0])
        indexed = matrix2[hero-1]  
        if indexed[1]==1:
            final_index_test.append(1)
        elif indexed[2]==1:
            final_index_test.append(2)
        elif indexed[3]==1:
            final_index_test.append(3)
    final_index_test = np.array(final_index_test)
    
    return train_data, final_index, test_data, final_index_test

In [24]:
train_data5, final_index5, test_data5, final_index_test5 = test_train(matrix4[0:50000], matrix5[0:50000])
train_data4, final_index4, test_data4, final_index_test4 = test_train(matrix41[0:50000], matrix51[0:50000])
train_data3, final_index3, test_data3, final_index_test3 = test_train(matrix42[0:50000], matrix52[0:50000])
train_data2, final_index2, test_data2, final_index_test2 = test_train(matrix43[0:50000], matrix53[0:50000])


In [25]:
import tensorflow as tf
#import keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [26]:
'''
    Started the neural networks implementation with an Embedding layer and they are followed by 2 layers of 
    bidirectional LSTMs followed by a dense layer with activation method softmax.
'''
# Build the model.
inputs5 = keras.Input(shape=(4,), dtype="int32")
x5 = layers.Embedding(40000, 4)(inputs5)
x5 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x5)
x5 = layers.Bidirectional(layers.LSTM(50))(x5)
outputs5 = layers.Dense(4, activation="softmax")(x5)
model5 = keras.Model(inputs5, outputs5)
model5.summary()


inputs4 = keras.Input(shape=(3,), dtype="int32")
x4 = layers.Embedding(40000, 4)(inputs4)
x4 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x4)
x4 = layers.Bidirectional(layers.LSTM(50))(x4)
outputs4 = layers.Dense(4, activation="softmax")(x4)
model4 = keras.Model(inputs4, outputs4)
model4.summary()


inputs3 = keras.Input(shape=(2,), dtype="int32")
x3 = layers.Embedding(40000, 4)(inputs3)
x3 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x3)
x3 = layers.Bidirectional(layers.LSTM(50))(x3)
outputs3 = layers.Dense(4, activation="softmax")(x3)
model3 = keras.Model(inputs3, outputs3)
model3.summary()


inputs2 = keras.Input(shape=(1,), dtype="int32")
x2 = layers.Embedding(40000, 4)(inputs2)
x2 = layers.Bidirectional(layers.LSTM(50, return_sequences=True))(x2)
x2 = layers.Bidirectional(layers.LSTM(50))(x2)
outputs2 = layers.Dense(4, activation="softmax")(x2)
model2 = keras.Model(inputs2, outputs2)
model2.summary()



Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 4, 4)              160000    
_________________________________________________________________
bidirectional_8 (Bidirection (None, 4, 100)            22000     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 404       
Total params: 242,804
Trainable params: 242,804
Non-trainable params: 0
_________________________________________________________________
Model: "functional_11"
_________________________________________________________________
Layer (type)             

In [27]:
# Compile the model.
model5.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model4.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model3.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model2.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [28]:
#model.fit(train_data, train_target, batch_size=30, epochs=100)
model5.fit(
  train_data5,
  to_categorical(final_index5),
  epochs=20,
  batch_size=32
)

model4.fit(
  train_data4,
  to_categorical(final_index4),
  epochs=20,
  batch_size=32
)


model3.fit(
  train_data3,
  to_categorical(final_index3),
  epochs=20,
  batch_size=32
)

model2.fit(
  train_data2,
  to_categorical(final_index2),
  epochs=20,
  batch_size=32
)

Epoch 1/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0532 - accuracy: 0.4409
Epoch 2/20
1250/1250 [==============================] - 6s 4ms/step - loss: 1.0113 - accuracy: 0.4854
Epoch 3/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0077 - accuracy: 0.4858
Epoch 4/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0058 - accuracy: 0.4910
Epoch 5/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0043 - accuracy: 0.4905
Epoch 6/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0027 - accuracy: 0.4913
Epoch 7/20
1250/1250 [==============================] - 5s 4ms/step - loss: 1.0014 - accuracy: 0.4933
Epoch 8/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9994 - accuracy: 0.4925
Epoch 9/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.9967 - accuracy: 0.4971
Epoch 10/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.995

1250/1250 [==============================] - 3s 2ms/step - loss: 0.9655 - accuracy: 0.5408


In [29]:
model5.evaluate(
  test_data5,
  to_categorical(final_index_test5)
)

model4.evaluate(
  test_data4,
  to_categorical(final_index_test4)
)

model3.evaluate(
  test_data3,
  to_categorical(final_index_test3)
)

model2.evaluate(
  test_data2,
  to_categorical(final_index_test2)
)

313/313 [==============================] - 0s 861us/step - loss: 0.9775 - accuracy: 0.5386


[0.9775075912475586, 0.5386000275611877]

In [30]:
predictions5 = model5.predict(test_data5)
predictions4 = model4.predict(test_data4)
predictions3 = model3.predict(test_data3)
predictions2 = model2.predict(test_data2)

In [32]:
#print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

In [33]:
y_pred5 = np.argmax(predictions5, axis=1)
y_pred4 = np.argmax(predictions4, axis=1)
y_pred3 = np.argmax(predictions3, axis=1)
y_pred2 = np.argmax(predictions2, axis=1)
y_pred5[0:20]

array([2, 2, 3, 2, 1, 1, 2, 1, 2, 2, 2, 2, 3, 2, 2, 1, 2, 3, 1, 3])

In [34]:
'''Self test is used. final_index_test5 contains the test target for 5th hero.
    Similarly, final_index_test4 is for 4th hero, final_index_test3 is for 3rd hero, 
    final_index_test4 is for 2nd hero.
'''
my_ytest5 = np.array(final_index_test5)
my_ytest4 = np.array(final_index_test4)
my_ytest3 = np.array(final_index_test3)
my_ytest2 = np.array(final_index_test2)

my_ytest_finalhero5 = my_ytest5
my_ytest_finalhero4 = my_ytest4
my_ytest_finalhero3 = my_ytest3
my_ytest_finalhero2 = my_ytest2

#my_ytest_finalhero
my_ypred5 = np.array(y_pred5)
my_ypred4 = np.array(y_pred4)
my_ypred3 = np.array(y_pred3)
my_ypred2 = np.array(y_pred2)

my_ypred_finalhero5 = my_ypred5
my_ypred_finalhero4 = my_ypred4
my_ypred_finalhero3 = my_ypred3
my_ypred_finalhero2 = my_ypred2

#my_ypred_finalhero
totals = my_ytest5.shape[0]
#totals
num_correct5 = sum(p == t for p, t in zip(my_ypred_finalhero5, my_ytest_finalhero5))
num_correct4 = sum(p == t for p, t in zip(my_ypred_finalhero4, my_ytest_finalhero4))
num_correct3 = sum(p == t for p, t in zip(my_ypred_finalhero3, my_ytest_finalhero3))
num_correct2 = sum(p == t for p, t in zip(my_ypred_finalhero2, my_ytest_finalhero2))

#num_correct
total5 = (num_correct5/totals)*100
total4 = (num_correct4/totals)*100
total3 = (num_correct3/totals)*100
total2 = (num_correct2/totals)*100

print("Prediction accuracy for 5th hero:", total5)
print("Prediction accuracy for 4th hero:", total4)
print("Prediction accuracy for 3rd hero:", total3)
print("Prediction accuracy for 2nd hero:", total2)

Prediction accuracy for 5th hero: 50.12
Prediction accuracy for 4th hero: 48.699999999999996
Prediction accuracy for 3rd hero: 47.49
Prediction accuracy for 2nd hero: 53.86


In [36]:
#print(final_index_test[10:20]) # [7, 2, 1, 0, 4]

In [37]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score
#testing table of heros
test1 = my_ytest5[5065:8623]
test2 = my_ytest5[4374:8026]
test3 = my_ytest5[4372:6746]
test4 = my_ytest5[5809: 9360]
test5 = my_ytest5[8477:9646]
test6 = my_ytest5[201:8306]
test7 = my_ytest5[4863:9545]
pred1 = my_ypred5[5065:8623]
pred2 = my_ypred5[4374:8026]
pred3 = my_ypred5[4372:6746]
pred4 = my_ypred5[5809: 9360]
pred5 = my_ypred5[8477:9646]
pred6 = my_ypred5[201:8306]
pred7 = my_ypred5[4863:9545]

score1 = f1_score(pred1, test1, average='weighted')
score2 = f1_score(pred2, test2, average='weighted')
score3 = f1_score(pred3, test3, average='weighted')
score4 = f1_score(pred4, test4, average='weighted')
score5 = f1_score(pred5, test5, average='weighted')
score6 = f1_score(pred6, test6, average='weighted')
score7 = f1_score(pred7, test7, average='weighted')

print("f1", score1, score2, score3, score4, score5, score6, score7)

score1 = accuracy_score(pred1, test1)
score2 = accuracy_score(pred2, test2)
score3 = accuracy_score(pred3, test3)
score4 = accuracy_score(pred4, test4)
score5 = accuracy_score(pred5, test5)
score6 = accuracy_score(pred6, test6)
score7 = accuracy_score(pred7, test7)

print("accuracy",score1, score2, score3, score4, score5, score6, score7)

score1 = precision_score(pred1, test1, average='weighted')
score2 = precision_score(pred2, test2, average='weighted')
score3 = precision_score(pred3, test3, average='weighted')
score4 = precision_score(pred4, test4, average='weighted')
score5 = precision_score(pred5, test5, average='weighted')
score6 = precision_score(pred6, test6, average='weighted')
score7 = precision_score(pred7, test7, average='weighted')

print("precision", score1, score2, score3, score4, score5, score6, score7)

f1 0.49559246071659235 0.5006041760809358 0.5095093812192859 0.4958338817822475 0.48812686934679483 0.5132141639867053 0.49445481357352
accuracy 0.48875772906127035 0.4947973713033954 0.5033698399326032 0.48943959448042806 0.47904191616766467 0.5052436767427514 0.4873985476292183
precision 0.510354533679983 0.5137449187384697 0.5223058963851867 0.5102410665573287 0.5085765632195265 0.5323451513761331 0.5099447506028093


In [38]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score
#testing table of heros
test1 = my_ytest4[5065:8623]
test2 = my_ytest4[4374:8026]
test3 = my_ytest4[4372:6746]
test4 = my_ytest4[5809: 9360]
test5 = my_ytest4[8477:9646]
test6 = my_ytest4[201:8306]
test7 = my_ytest4[4863:9545]
pred1 = my_ypred4[5065:8623]
pred2 = my_ypred4[4374:8026]
pred3 = my_ypred4[4372:6746]
pred4 = my_ypred4[5809: 9360]
pred5 = my_ypred4[8477:9646]
pred6 = my_ypred4[201:8306]
pred7 = my_ypred4[4863:9545]

score1 = f1_score(pred1, test1, average='weighted')
score2 = f1_score(pred2, test2, average='weighted')
score3 = f1_score(pred3, test3, average='weighted')
score4 = f1_score(pred4, test4, average='weighted')
score5 = f1_score(pred5, test5, average='weighted')
score6 = f1_score(pred6, test6, average='weighted')
score7 = f1_score(pred7, test7, average='weighted')

print("f1", score1, score2, score3, score4, score5, score6, score7)

score1 = accuracy_score(pred1, test1)
score2 = accuracy_score(pred2, test2)
score3 = accuracy_score(pred3, test3)
score4 = accuracy_score(pred4, test4)
score5 = accuracy_score(pred5, test5)
score6 = accuracy_score(pred6, test6)
score7 = accuracy_score(pred7, test7)

print("accuracy",score1, score2, score3, score4, score5, score6, score7)

score1 = precision_score(pred1, test1, average='weighted')
score2 = precision_score(pred2, test2, average='weighted')
score3 = precision_score(pred3, test3, average='weighted')
score4 = precision_score(pred4, test4, average='weighted')
score5 = precision_score(pred5, test5, average='weighted')
score6 = precision_score(pred6, test6, average='weighted')
score7 = precision_score(pred7, test7, average='weighted')

print("precision", score1, score2, score3, score4, score5, score6, score7)

f1 0.49034718453444137 0.4963610145207944 0.5099708820320165 0.4833039219541743 0.4864334929509075 0.49094308042659024 0.4894777955493711
accuracy 0.4876335019673974 0.4931544359255203 0.504212299915754 0.48070965925091524 0.48246364414029086 0.48883405305367056 0.486544211875267
precision 0.5004149063846391 0.5082311170576838 0.5291903064946226 0.49288465444622565 0.49787869221851694 0.499946904912146 0.5000102341013679


In [39]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score
#testing table of heros
test1 = my_ytest3[5065:8623]
test2 = my_ytest3[4374:8026]
test3 = my_ytest3[4372:6746]
test4 = my_ytest3[5809: 9360]
test5 = my_ytest3[8477:9646]
test6 = my_ytest3[201:8306]
test7 = my_ytest3[4863:9545]
pred1 = my_ypred3[5065:8623]
pred2 = my_ypred3[4374:8026]
pred3 = my_ypred3[4372:6746]
pred4 = my_ypred3[5809: 9360]
pred5 = my_ypred3[8477:9646]
pred6 = my_ypred3[201:8306]
pred7 = my_ypred3[4863:9545]

score1 = f1_score(pred1, test1, average='weighted')
score2 = f1_score(pred2, test2, average='weighted')
score3 = f1_score(pred3, test3, average='weighted')
score4 = f1_score(pred4, test4, average='weighted')
score5 = f1_score(pred5, test5, average='weighted')
score6 = f1_score(pred6, test6, average='weighted')
score7 = f1_score(pred7, test7, average='weighted')

print("f1", score1, score2, score3, score4, score5, score6, score7)

score1 = accuracy_score(pred1, test1)
score2 = accuracy_score(pred2, test2)
score3 = accuracy_score(pred3, test3)
score4 = accuracy_score(pred4, test4)
score5 = accuracy_score(pred5, test5)
score6 = accuracy_score(pred6, test6)
score7 = accuracy_score(pred7, test7)

print("accuracy",score1, score2, score3, score4, score5, score6, score7)

score1 = precision_score(pred1, test1, average='weighted')
score2 = precision_score(pred2, test2, average='weighted')
score3 = precision_score(pred3, test3, average='weighted')
score4 = precision_score(pred4, test4, average='weighted')
score5 = precision_score(pred5, test5, average='weighted')
score6 = precision_score(pred6, test6, average='weighted')
score7 = precision_score(pred7, test7, average='weighted')

print("precision", score1, score2, score3, score4, score5, score6, score7)

f1 0.5530389372116626 0.5519500055554167 0.5614124245599575 0.546581833542907 0.5457973893467024 0.5473585438571409 0.5543421687565435
accuracy 0.48173130972456435 0.4789156626506024 0.4877843302443134 0.47479583215995497 0.4704875962360992 0.47661937075879085 0.4824861170439983
precision 0.6529412931971282 0.6543686692046056 0.6654902589423122 0.6480543084573193 0.6563596474626766 0.6440622388473425 0.6550475161003082


In [40]:
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score
#testing table of heros
test1 = my_ytest2[5065:8623]
test2 = my_ytest2[4374:8026]
test3 = my_ytest2[4372:6746]
test4 = my_ytest2[5809: 9360]
test5 = my_ytest2[8477:9646]
test6 = my_ytest2[201:8306]
test7 = my_ytest2[4863:9545]
pred1 = my_ypred2[5065:8623]
pred2 = my_ypred2[4374:8026]
pred3 = my_ypred2[4372:6746]
pred4 = my_ypred2[5809: 9360]
pred5 = my_ypred2[8477:9646]
pred6 = my_ypred2[201:8306]
pred7 = my_ypred2[4863:9545]

score1 = f1_score(pred1, test1, average='weighted')
score2 = f1_score(pred2, test2, average='weighted')
score3 = f1_score(pred3, test3, average='weighted')
score4 = f1_score(pred4, test4, average='weighted')
score5 = f1_score(pred5, test5, average='weighted')
score6 = f1_score(pred6, test6, average='weighted')
score7 = f1_score(pred7, test7, average='weighted')

print("f1", score1, score2, score3, score4, score5, score6, score7)

score1 = accuracy_score(pred1, test1)
score2 = accuracy_score(pred2, test2)
score3 = accuracy_score(pred3, test3)
score4 = accuracy_score(pred4, test4)
score5 = accuracy_score(pred5, test5)
score6 = accuracy_score(pred6, test6)
score7 = accuracy_score(pred7, test7)

print("accuracy",score1, score2, score3, score4, score5, score6, score7)

score1 = precision_score(pred1, test1, average='weighted')
score2 = precision_score(pred2, test2, average='weighted')
score3 = precision_score(pred3, test3, average='weighted')
score4 = precision_score(pred4, test4, average='weighted')
score5 = precision_score(pred5, test5, average='weighted')
score6 = precision_score(pred6, test6, average='weighted')
score7 = precision_score(pred7, test7, average='weighted')

print("precision", score1, score2, score3, score4, score5, score6, score7)

f1 0.6910164387205394 0.7006561495053676 0.7196420232484526 0.6886882791476033 0.6931869221985927 0.6939515301975933 0.6942248488552786
accuracy 0.5356942102304666 0.5465498357064622 0.5711878685762426 0.5333708814418474 0.5466210436270317 0.5376927822331894 0.5405809483126869
precision 0.9866954174839184 0.9876623519475359 0.9864722010764273 0.9861351300875094 0.9757156240483722 0.9893832762018485 0.9858192396526199
